In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

##plotting modules
import seaborn as sns

In [ ]:
%matplotlib inline

##### Set General Parameters

In [ ]:
MFr_O2 = 0.2095 #Molar fraction of O2 in ambient air
MR_O2_In = 0.265 #Molar ratio O2/Inerts (Inerts are CO2, N2 and all other components having a fized conc in ambient)
Patm_ref = 1013.25 #Reference ambient pressure
DOss_20 = 9.08 #Referene DO saturation concentration at 20 deg C
tetha = 1.024 #for temperature adjustm

Vel_BaseScale = 2 #base scale velocity for anemomenter reading 
Eo = 1
Efs = 2 #Param of anemometer 

Tube_diam = 0.0381 #diameter of tube driving the off-gas throught the anemomenter
Tube_area = np.pi*(Tube_diam/2)**2

#Points = 9 #Number of points on the tank

#### This notebook helps in managing data from the old DICEA off-gas analyzer

##### Insert path to data files

In [ ]:
os.chdir(r'../data/analyzer/')

In [ ]:
ls

In [ ]:
path = os.getcwd()

In [ ]:
file_names = []
data_frames = []

for filename in os.listdir(path):
    name = os.path.splitext(filename)[0]
    file_names.append(name)
    df = pd.read_csv(path + '//' + filename, delimiter='\t', sep='.', header=None)
    df.rename(columns={0: 'Time (s)', 1: name + ' Flow (v)', 2: name + ' O2 (v)', 3: name + ' Press (v)'}, inplace=True)
    data_frames.append(df)

combined = pd.concat(data_frames, axis=1)
combined = combined.T.groupby(level=0).first().T

##### Separate O2 and Air flow V signal data frame

In [ ]:
combined_clean = combined.set_index(['Time (s)'], drop=True)
O2v = combined_clean[[col for col in combined_clean.columns if 'O2' in col]]
Flowv = combined_clean[[col for col in combined_clean.columns if 'Flow' in col]]
#Presv = combined_clean[[col for col in combined_clean.columns if 'Press' in col]] #not in use

##### Control the status of the txt files present in the folder for O2 signal

In [ ]:
plt.figure();
O2v[100:].plot(subplots=True, figsize=(15, 30), style='o');

##### O2 stats

In [ ]:
O2v_stats = []
for f in file_names:
    df = pd.DataFrame.describe(pd.DataFrame(O2v[f + ' O2 (v)'][100:])) # length set to [100:] to skip sensor stabilization
    O2v_stats.append(df)
O2v_stats = pd.concat(O2v_stats, axis=1)

##### Air flow stats

In [ ]:
Flowv_stats = []
for f in file_names:
    df = pd.DataFrame.describe(pd.DataFrame(Flowv[f + ' Flow (v)'][100:])) #air flow does not need stabilization time but we use the same as for O2sensor
    Flowv_stats.append(df)
Flowv_stats = pd.concat(Flowv_stats, axis=1)

##### Check visually O2 and Flow statistics for each file

In [ ]:
O2v_stats

In [ ]:
xlabels = []
for k in range(O2v.columns.shape[0]): 
    xlabels = np.append(xlabels, O2v.columns[k][5:18])
xlabels = xlabels.tolist()

In [ ]:
O2v

In [ ]:
O2v[100:].boxplot(figsize=(12,5));

#### Signal translate

##### Store reference O2 value

In [ ]:
Reference = O2v_stats['test_16-11-22_1137_001 O2 (v)'][1] # store mean of reference file
Referencesd = O2v_stats['test_16-11-22_1137_001 O2 (v)'][2] # store std of reference file

##### OTE calculation

In [ ]:
yO2 = O2v[100:] / Reference * MFr_O2 #molar fraction in the off-gas
YO2 = yO2 / (1 - yO2) #molar ratio O2 inerts in the off-gas NOTE: here the CO2 is considered to be completely absorbed

OTE = (MR_O2_In - YO2) / MR_O2_In * 100
OTE_mean = pd.DataFrame(OTE.mean())
OTE_sd = pd.DataFrame(OTE.std())

In [ ]:
ax = OTE_mean.plot(title='Stationary measurement', figsize=(12,3), yerr=OTE_sd, legend=False, style='^');

ax.set_xlabel('Time (HH:MM)')
ax.set_ylabel('OTE (%)')

ax.set_xticks(range(len(file_names)))
ax.set_xticklabels(['Point 1', 'Point 2', 'Point 3', 'Point 4', 'Point 5', 
                    'Point 6', 'Point 7', 'Point 8', 'Point 9', 'Point 10'], 
                   rotation=45);
# plt.savefig('Stationary measurement')

##### Write stats to xlsx file

<div class="alert alert-success">
    <b> EXERCISE: </b> try to save your data adjusting the code below
</div>

In [ ]:
# to which file to add the following dataframes
# writer = pd.ExcelWriter(path + file_names[0][5:7] + file_names[0][8:10] + file_names[0][11:13] + '_OldAn' + '.xlsx') 

# O2v.to_excel(writer, 'O2') #what to write to which sheet
# Flowv.to_excel(writer, 'Flow')
# O2v_stats.to_excel(writer, 'Stats O2')
# Flowv_stats.to_excel(writer, 'Stats Flow')
# OTE.to_excel(writer, 'OTE')
# OTE_mean.to_excel(writer, 'Stats OTE', header=['mean OTE'])
# OTE_sd.to_excel(writer, 'Stats OTE', startcol=3, header=['StDev OTE'])
# writer.save()